# Overview

Terrain is the web API that the CyVese Discovery Environment uses to perform all of its tasks. Terrain is a REST-like web API. That is, it acts like a traditional web server in many ways. Requests that are sent to terrain are formatted just like a request that your browser might make in order to open a web page. Responses from Terrain are typically very similar to responses that your browser might receive from a request. The primary difference is the response format. The format that your browser receives from a web server will typically be formatted as a a web page whereas a response from Terrain will usually be a document formatted in JavaScript Object Notation (JSON). The primary reason for this difference is that JSON is easier for computers to read than HTML, and it's easier for humans to read than another common format in use on the internet, XML. If you're unfamiliar with JSON, DigitalOcean has this great [introduction](https://www.digitalocean.com/community/tutorials/an-introduction-to-json) that will quickly bring you up to speed.

Without further ado, let's give Terrain a try! 

### Prerequisites

Before we can use Terrain, we'll need some prerequisites. The `requests` library is an easy way to make requests to web services in Python, so let's import it so that we can use it later. It'll also be necessary to prompt for a password and to pretty-print some data structures.

In [ ]:
import getpass
import json
import pprint
import requests

### The Welcome Message

Terrain has a bit of a goofy welcome message that you can see by hitting its root endpoint. The original purpose of the message was for developers to have an easy way to verify that Terrain was in fact running and responsive. We still use this endpoint on a regular basis, but now the calls to it are automated. For some reason, the computers don't find it as amusing as the humans do.

In [ ]:
r = requests.get("https://de.cyverse.org/terrain/")
r.raise_for_status()
print(r.text)

### Authentication

To do anything interesting in terrain, you have to authenticate to the service. Terrain uses [Keycloak](https://www.keycloak.org/) for authentication. Typically you'd need to regsiter an OpenID Connect (OIDC) client to allow you to authenticate directly to Keyclok, which can be a lot of work. For this reason, Terrain has an endpoint that you can use to obtain a token indirectly using Terrain's own OIDC client. But in order to get the access token, we'll need your CyVerse username and password.

In [ ]:
print("Username: ", end='', flush=True)
username = input()
print("Password: ", end='', flush=True)
password = getpass.getpass()

And now we can obtain an access token.

In [ ]:
r = requests.get("https://de.cyverse.org/terrain/token/keycloak", auth=(username, password))
r.raise_for_status()
token = r.json()['access_token']
auth_headers = {"Authorization": "Bearer " + token}

# Launching Apps using Terrain

Terrain currently supports three different types of apps: Interactive (also commonly known as VICE) apps, Executable apps, and High Performance Computing (HPC) apps. As the name implies, interactive apps allow you to log in and work with the software and data directly. This is useful when you need to visualize data or explore your data directly. Executable apps run in the background, perform their tasks and email you when they're finished. This is most useful when you have a lot of data to process and you already know how you want to analyze it. HPC apps run on the supercomputers at the Texas Advanced Computing Center using the TAPIS API. These apps also run in the background and email you when they're finished, but because they run on supercomputers, they can frequently handle larger workloads.

## Launching Interactive (VICE) Apps

The acronym, VICE, stands for Visual Interactive Computing Environment. The most important thing to know about these apps is that they let you work with the software and your data directly, which is great for exploration. The first step in launching a VICE app is finding the app that you want to launch. One of the more commonly used VICE apps is JupyterLab, so we'll search for a suitable app.

In [ ]:
query_params = {"search": "JupyterLab Datascience"}
r = requests.get("https://de.cyverse.org/terrain/apps", headers=auth_headers, params=query_params)
r.raise_for_status()
pprint.pprint(r.json())

There are several `JupyterLab Datascience` apps, so we'll want to make sure that we get the correct one. In this case, we'll want to filter the list of matching analyses for the exact name, so let's do that.

In [ ]:
app = [ a for a in r.json()['apps'] if a['name'] == 'JupyterLab Datascience' ][0]
pprint.pprint(app)

### Getting Analysis Submission Information

Before you can submit an analysis to the Discovery environment, you'll need to know how to pass parameters to the app. The endpoint to get information on how to submit an analysis for a specific app is GET /terrain/apps/{system-id}/{app-id}, where the System ID and app ID are fields obtained from the analysis listing returned by the previous endpoint. We grabbed a copy of the app information from the app listing above, so we can grab the system ID and app ID from it.

In [ ]:
system_id = app["system_id"]
app_id = app["id"]

url = "https://de.cyverse.org/terrain/apps/{0}/{1}".format(system_id, app_id)
r = requests.get(url, headers=auth_headers)
r.raise_for_status()
pprint.pprint(r.json())

You may nave noticed that this app has several versions. Having multiple versions of a single app allows us to keep up with new software versions as they come out without having to create a brand new app for each software version. If you want to launch a specific version of the app, you can pick the version that you want then obtain the submission information for that specific version. Suppose for example, we wanted to submit version 4.0.1 of this app. The first step in doing that is to obtain the app submission information for that specific app version.

In [ ]:
app_version_id = [ v["version_id"] for v in r.json()["versions"] if v["version"] == "4.0.1" ][0];
print("Version ID:", app_version_id)

Now that we've got the ID for that specific version, we can get the analysis submission information for that specific version.

In [ ]:
url = "https://de.cyverse.org/terrain/apps/{0}/{1}/versions/{2}".format(system_id, app_id, app_version_id)
r = requests.get(url, headers=auth_headers)
r.raise_for_status()
pprint.pprint(r.json())

### Building the Analysis Submission Request Body

The analysis submission endpoint is the first endpoint we've encountered so far that has a request body, and this request body needs to be formatted correctly for the analysis submission to succeed. The request body looks something like this:

``` json
{
  "config": {},
  "name": "string",
  "app_id": "string",
  "app_version_id": "string",
  "system_id": "string",
  "debug": false,
  "output_dir": "string",
  "notify": true,
  "requirements": [
    {
        "min_cpu_cores": integer,
        "max_cpu_cores": integer,
        "min_memory_limit": integer,
        "min_disk_space": integer,
        "step_number": integer
    }
  ]
}
```

I've taken the liberty of removing optional fields from the example. The fields are defined as follows:

| Parameter Name   | Description                                                                       |
| --------------   | --------------------------------------------------------------------------------- |
| config           | A map from parameter ID to parameter value.                                       |
| name             | The name of the analysis.                                                         |
| app_id           | The app ID from the submission information above.                                 |
| system_id        | The system ID from the submission information above.                              |
| debug            | This parameter can be used to enable debugging, which isn't necessary.            |
| output_dir       | The path to the folder in the data store where the output files should be placed. |
| notify           | This parameter can be used to enable or disable job status update notifications.  |
| requirements     | This parameter contains resource requirements needed to run the analysis.         |
| min_cpu_cores    | The minimum number of CPU cores needed to run the analysis.                       |
| max_cpu_cores    | The maximum number of CPU cores that can be used simultaneously for the analysis. |
| min_memory_limit | The minimum amount of RAM required to run the analysis.                           |
| min_disk_space   | The minimum amount of disk space needed to perform the analysis.                  |
| step_number      | The sequential number of the step in an analysis (more on this later).            |

So now we have to plug in the values. In this case, there are no parameters to specify, so the configuration can be left empty.

One important caveat for interactive analyses is that they run until you explicitly stop them or they time out. For this reason, it's very important to limit the amount of resources that you request in order to avoid exhausting your compute allocation. When you're submitting the analysis, it's useful to set the minimum and maximum CPU cores to the number of cores you need to be able to use simultaneously in order to complete your work. In this case, we'll request just a single core. It's also a good practice to specify the minimum amounts of RAM and disk space that you'll need to finish your work.

Note that the request body below is written in Python instead of JSON, so this will look a little bit different from the example request body above.

In [ ]:
request_body = {
    "config": {},
    "name": "Example VICE Analysis",
    "app_id": app_id,
    "app_version_id": app_version_id,
    "system_id": system_id,
    "debug": False,
    "output_dir": "/iplant/home/" + username + "/analyses",
    "notify": True,
    "requirements": [
        {
            "min_cpu_cores": 1,
            "max_cpu_cores": 1,
            "min_memory_limit": 2 * 2 ** 30,
            "min_disk_space": 8 * 2 ** 30,
            "step_number": 0,
        },
    ],
}
pprint.pprint(request_body)

One thing to note here is that the requirements field contains a list of objects. This can be somewhat counterintuitive at first, but the reason that this is done is because apps in the Discovery Environment can have muliple steps and the tool associated with each step in the analysis may require different amounts of resources. To determine how many objects to include in the list of requirements, you can refer to the app submission information from the prevous API call. The number of objects should be the same in the submission request body. The step number in each of these objects correlates to the sequential number of the step (starting at `0`). So an analysis pipeline wiht three steps will have step numbers `0`, `1`, and `2`.

### Submitting the Analysis

Now that the request body has been built, the analysis can be submitted. This can be done by sending an HTTP `POST` request containing the request body we created above to the `/analyses` endpoint in Terrain.

In [ ]:
r = requests.post("https://de.cyverse.org/terrain/analyses", headers=auth_headers, json=request_body)
r.raise_for_status()
pprint.pprint(r.json())

We're also going to need the analysis ID in the next step, so it's a good idea to save it into a separate variable for later use.

In [ ]:
analysis_id = r.json()["id"]
pprint.pprint(analysis_id)

### Connecting to the Analysis

Whenever you launch an interactive analysis, the DE generates a unique URL that you can use to connect to that analysis. In order to be able to connect to the analysis, it's first necessary to obtain the URL that the DE generated. Fortuanately, the URL is returned by the analysis listing endpoint, which is capable of returning information about an analysis with a specific ID.

In [ ]:
analysis_filters = json.dumps([{"field": "id", "value": analysis_id}])
query_params = {"filter": analysis_filters}
r = requests.get("https://de.cyverse.org/terrain/analyses", headers=auth_headers, params=query_params)
r.raise_for_status()
pprint.pprint(r.json())

The field that we're most interested in here is `interactive_urls`. This field contains the URL that we can use to connect to each step in the analysis. At this point in time, VICE apps are limited to just one step, so you can be sure that there will be just one URL in the list.

In [ ]:
interactive_url = r.json()["analyses"][0]["interactive_urls"][0]
print(interactive_url)

If you click on the URL displayed by the previous cell, it will connect you to the app, prompting you to log in if you don't have an active DE session. If you're already logged into the DE, you won't have to log in again.

### Shutting Down the Analysis

When you're done working with an interactive analysis, it's important to shut it down so that you don't unnecessarily consume CPU hours in your compute allocation. You can shut down an analysis using the analysis termination endpoint.

In [ ]:
url = "https://de.cyverse.org/terrain/analyses/{0}/stop".format(analysis_id)
r = requests.post(url, headers=auth_headers)
r.raise_for_status()
pprint.pprint(r.json())

## Launching Executable Apps

Our executable apps perform their tasks in the background and let you know when they're done so that you don't have to keep an eye on them. This can be very useful for large tasks that require a lot of data analysis such as genome sequence alignments.

### Searching for Apps

For this example, suppose you've got an eight-hundred-word essay to write, and you want to use the DE to make sure that your essay is long enough. You can use one of the DE's word count apps to find out!

In [ ]:
query_params = {"search": "DE Word Count"}
r = requests.get("https://de.cyverse.org/terrain/apps", headers=auth_headers, params=query_params)
r.raise_for_status()
pprint.pprint(r.json())

### Getting Analysis Submission Information

Before you can submit an analysis to the Discovery environment, you'll need to know how to pass parameters to the app. The endpoint to get information on how to submit an analysis for a specific app is `GET /terrain/apps/{system-id}/{app-id}`, where the System ID and app ID are fields obtained from the analysis listing returned by the previous endpoint. Since we still have the response body from the app lookup, we can extract the information we need from it.

In [ ]:
app_listing = r.json()["apps"][0]
system_id = app_listing["system_id"]
app_id = app_listing["id"]
print("System ID: ", system_id)
print("App ID: ", app_id)

Now calling the analysis submission information endpoint gives us the information we need.

In [ ]:
url = "https://de.cyverse.org/terrain/apps/{0}/{1}".format(system_id, app_id)
r = requests.get(url, headers=auth_headers)
r.raise_for_status()
pprint.pprint(r.json())

The output from this endpoint deserves a little explanation. At the top level, we have the basic app information such as the name, ID, and description of the app. The top level also contains a list labeled `groups`. These groups provide a way to place related parameters on the same panel in the app launch window in the DE. Each group contains a list of parameters, and the parameters themselves provide the information we need to submit the job.

The primary piece of information that we're going to need from this file is the parameter ID for the input file name. We may as well grab it now.

In [ ]:
parameter_id = r.json()["groups"][0]["parameters"][0]["id"]
print("Parameter ID: ", parameter_id)

### Building the Analysis Submission Request Body

The analysis submission endpoint is the first endpoint we've encountered so far that has a request body, and this request body needs to be formatted correctly for the analysis submission to succeed. The request body looks something like this:

``` json
{
  "config": {},
  "name": "string",
  "app_id": "string",
  "system_id": "string",
  "debug": false,
  "output_dir": "string",
  "notify": true
}
```

I've taken the liberty of removing optional fields from the example. The fields are defined as follows:

| Parameter Name | Description                                                                       |
| -------------- | --------------------------------------------------------------------------------- |
| config         | A map from parameter ID to parameter value.                                       |
| name           | The name of the analysis.                                                         |
| app_id         | The app ID from the submission information above.                                 |
| system_id      | The system ID from the submission information above.                              |
| debug          | This parameter can be used to enable debugging, which isn't necessary.            |
| output_dir     | The path to the folder in the data store where the output files should be placed. |
| notify         | This parameter can be used to enable or disable job status update notifications.  |

So now we have to plug in the values. Suppose the essay whose words we have to count is at this path in the CyVerse data store: `/iplant/home/shared/workshop_material/terrain_intro/essay.txt`. And now we have enough information to format the request body.

Keep in mind that the request body below is written in Python rather than JSON, so it will look slightly different from the JSON request body listed above. The `requests` library will convert this Python object to a JSON object for us before sending the request to terrain.

In [ ]:
request_body = {
    "config": {
        parameter_id: "/iplant/home/shared/workshop_material/terrain_intro/essay.txt"
    },
    "name": "essay-word-count",
    "app_id": app_id,
    "system_id": system_id,
    "debug": False,
    "output_dir": "/iplant/home/" + username + "/analyses",
    "notify": True
}
pprint.pprint(request_body)

Now we can finally submit the request.

In [ ]:
r = requests.post("https://de.cyverse.org/terrain/analyses", headers=auth_headers, json=request_body)
r.raise_for_status()
pprint.pprint(r.json())

### Listing Analyses

At some point, you're going to want to be able to obtain a list of the analyses that you've submitted along with the status of each analysis. Fortunately obtaining this list is very easy. Note that we're limiting the number of analyses in the list to 5 in order to keep the output manageable. It's safe to do that because this endpoint sorts analyses in reverse order by start date.

In [ ]:
r = requests.get("https://de.cyverse.org/terrain/analyses?limit=5", headers=auth_headers)
r.raise_for_status()
pprint.pprint(r.json())

## Launching HPC Apps

Coming soon!

## Finding More Information

At this point, it would be reasonable to ask where you might find more information about the Terrain API. The good news is that we have interactive [Swagger documentation](https://de.cyverse.org/terrain/docs/). The bad news is that this is a work in progress. Depending on how long it's been since our production deployment was updated, you _may_ be able to find a little more information in our [QA deployment swagger documentation](https://qa.cyverse.org/terrain/docs/), but keep in mind that the API in our QA deployment may not exactly match the API in our production deployment.

If you can't find the documentation that you need in our Swagger documentation, you should be able to find the information that you need in our [old API documentation](https://cyverse-de.github.io/api/). The [endpoint index](https://cyverse-de.github.io/api/endpoint-index.html) contains links to all of the documented API endpoints.

Finally, you can always log into the Discovery Environment and ask us for help using the Intercom chat widget in the lower right-hand corner.